---
## Step 0: Load & Setup The Data

In [1]:
# general python packages
from __future__ import print_function
import os 
import glob
import math
import numpy as np
import pandas as pd
import random as ran
import matplotlib.pyplot as plt

# Deep Learning packages
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv3D, MaxPooling3D, AveragePooling3D, Input, ZeroPadding3D
from keras.layers.normalization import BatchNormalization
from keras import initializers
from keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, Adadelta, RMSprop, Adamax, Nadam, SGD # using Adam in this model, but just showing the other options here for future reference
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
# neuroimaging-specific python packages
import nilearn
from nilearn import plotting
import nibabel as nib

/home/slab/environments/OHBM_DL/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
cd /net/parasite/CMI/OHBM_DL_data/

/net/parasite/CMI/OHBM_DL_data


In [5]:
X_train = np.load('/net/parasite/CMI/OHBM_DL_data/train.npy')

In [6]:
X_test = np.load('/net/parasite/CMI/OHBM_DL_data/test.npy')

In [7]:
y_train = np.load('/net/parasite/CMI/OHBM_DL_data/train_labels.npy')

In [8]:
y_test = np.load('/net/parasite/CMI/OHBM_DL_data/test_labels.npy')

In [9]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]
image_shape = X_train[0].shape

In [10]:
print("Number of training examples =", n_train)
print ("Number of test examples", n_test)
print("Image data shape =", image_shape)

Number of training examples = 724
Number of test examples 182
Image data shape = (121, 145, 121, 3)


In [11]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_CMI_T1w_age_6-1-19'

# Step 1: Model Construction

In [12]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [13]:
model = Sequential() # The simplest model, a linear stack of layers

model.add(Conv3D(filters=64,
                 kernel_size=(3,3,3), #determines the width, height, depth of the 3D convolution window
                 activation='elu', #Exponential Linear Unit
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform', 
                 input_shape=image_shape)) # only the first layer needs to be told this info
model.add(Conv3D(filters=64, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(Conv3D(filters=64, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(MaxPooling3D((2,2,2),strides=(2,2,2))) # pooling is also referred to as a downsampling layer
model.add(BatchNormalization()) # Normalize the activations of the previous layer at each batch (aka make the mean activation close to 0 and the activation standard deviation close to 1)

model.add(Conv3D(filters=32, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(Conv3D(filters=32, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(MaxPooling3D((2,2,2),strides=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(filters=16, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(Conv3D(filters=16, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(MaxPooling3D((2,2,2),strides=(2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(filters=8, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(Conv3D(filters=8, kernel_size=(3,3,3), activation='elu', strides=(1,1,1), padding='same'))
model.add(MaxPooling3D((2,2,2),strides=(2,2,2)))
model.add(BatchNormalization())

model.add(AveragePooling3D((2,2,2),strides=(2,2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu',name='features')) #convert the output of the convolutional part of the CNN into a 1D feature vector. Length of vector = n_classes
model.add(Dense(1)) # final output is a single number (Age in this model)
model.summary()

filename="best_weights.h5"
filename2="weights.{epoch:02d}-{val_loss:.2f}.hdf5"


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 121, 145, 121, 64) 5248      
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 121, 145, 121, 64) 110656    
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 121, 145, 121, 64) 110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 60, 72, 60, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 72, 60, 64)    256       
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 60, 72, 60, 32)    55328     
_________________________________________________________________
conv

In [14]:
checkpoints = []

if not os.path.exists('Results00/'):
    os.makedirs('Results00/')

checkpoints.append(ModelCheckpoint('Results00/'+filename, 
                                   monitor='val_loss', 
                                   verbose=1, 
                                   save_best_only=True, 
                                   save_weights_only=True, 
                                   mode='auto', 
                                   period=1))

checkpoints.append(ModelCheckpoint('Results00/'+filename2, 
                                   monitor='val_loss', 
                                   verbose=1, 
                                   save_best_only=False, 
                                   save_weights_only=True, 
                                   mode='auto', 
                                   period=20))

checkpoints.append(TensorBoard(log_dir='Results00/TensorBoardLogs', 
                               histogram_freq=0, 
                               write_graph=True, 
                               write_images=False, 
                               embeddings_freq=0, 
                               embeddings_layer_names=['features'], 
                               embeddings_metadata='metadata.tsv'))

checkpoints.append(EarlyStopping(monitor='val_loss', mode='min', min_delta=0, patience=10))
checkpoints.append(ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0))
checkpoints.append(CSVLogger('Results00/log.csv'))

In [15]:
model.compile(loss='mse', # the objective that the model will try to minimize
              optimizer='adam', 
              metrics=['mae', 'acc']) # add in any other metrics you want to use to show performance of the model

**Saving/loading whole models (architecture + weights + optimizer state):**
1. the architecture of the model, allowing to re-create the model
2. the weights of the model
3. the training configuration (loss, optimizer)
4. the state of the optimizer, allowing to resume training exactly where you left off


# Step 2: Model Training

In [16]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [19]:
NUM_EPOCHS = 10 # defines for how many times the training will repeat. 1 epoch is 1 forward pass and 1 backward pass over all the training examples
BATCH_SIZE=6 # the number of training examples in one forward/backward pass (or for 1 epoch)
history1= model.fit(X_train, y_train, 
          validation_split=0.1,
          batch_size= BATCH_SIZE, 
          epochs = NUM_EPOCHS,
          callbacks=checkpoints)

Train on 651 samples, validate on 73 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[6,64,121,145,121] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/max_pooling3d_1/MaxPool3D_grad/MaxPool3DGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
log_dir ='Results00/'
from os.path import exists, join
with open(join(log_dir, 'metadata.tsv'), 'w') as f:
    np.savetxt(f, y_test)

# Step 3: Model Testing

In [ ]:
model.evaluate(x=x_test, y=y_test, callbacks=None)

# Step 4: Model Evaluation

In [ ]:
model.predict_on_batch(x)

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()